In [ ]:
!pip3 install transformers
!pip3 install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.4 MB/s 
     |████████████████████████████████| 7.6 MB 28.9 MB/s 
     |████████████████████████████████| 182 kB 44.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None


def generate(
        model,
        tokenizer,
        prompt,
        entry_count=1,
        entry_length=50,  # maximum number of words
        top_p=0.1,
        temperature=1,
):
    model.eval()
    generated_num = 0
    generated_list = []
    filter_value = -float("Inf")
    with torch.no_grad():
        for entry_idx in trange(entry_count):
            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
            

            for i in range(entry_length):
                outputs = model(generated, labels=generated)


                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)
                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:
                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
                output_list = list(generated.squeeze().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
                generated_list.append(output_text)

    return generated_list


# Function to generate multiple sentences. Test data should be a dataframe
def text_generation(lst):
    generated_lyrics = []
    for row in lst:
        x = generate(model, tokenizer, row)[0]
        x = x.strip("<|endoftext|>")
        try:
            generated_lyrics.append(x[len(row):].split(".")[0])
        except:
            generated_lyrics.append(x[len(row):])

    return generated_lyrics

def getDF(file_name):
    df = pd.read_csv(file_name)
    df["intput"] = df["sentence1"]+df["sentence2"]+df["sentence3"]+df["sentence4"]
    df = df.dropna(axis="rows")

    df.columns = df.columns.str.strip()
    return df

def add_end_token(lst):
    for i in range(len(lst)):
        lst[i] = lst[i]+'<|endoftext|>'
    return lst

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model =  GPT2LMHeadModel.from_pretrained('gpt2-medium')

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [ ]:
df = getDF('/content/drive/MyDrive/Corpus/for_human_evaluation.csv')
# df = getDF('train1000.csv')

# df = df.head(10)


lst = add_end_token(list(df['intput']))
# lst = list(df['intput'])

In [ ]:
generated_sentences = text_generation(lst)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(generated_sentences)

10

In [ ]:
generated_sentences

['The man was a very nice man',
 'The man was driving a truck',
 'The man was a man of great courage',
 'I was walking home from work',
 'The man was driving a truck',
 'The man was a young man',
 'The next day, the police arrived',
 'The new president of the United States is not a man of the people',
 'I was going to go to a movie with my friends',
 'I was walking home from work']

In [ ]:
df['generated_sentence'] = generated_sentences
# df = df.drop(['intput'],axis=1)
# df.to_csv("hello.csv")